In [1]:
import pandas as pd
database = pd.read_excel('database_TCC.xlsx')
database = database[['A','E','N','Rt']]
database

,A,E,N,Rt
0,25,1,11,1.638
1,25,1,5,2.595
2,25,2,8,1.976
3,25,3,11,4.164
4,25,3,5,2.869
5,35,1,8,1.421
6,35,2,11,1.564
7,35,2,5,2.101
8,35,3,8,1.591
9,25,1,8,1.946


In [2]:
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()
standardscaler.fit(database)
data = standardscaler.transform(database)
database = pd.DataFrame(data,columns=database.columns)

In [3]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,max_error

def split_x_and_y(database,x,y):
  dataset_x = database[x]
  dataset_y = database[y]
  return dataset_x,dataset_y


def scores(Y_true, Y_predicted):
  r2 = r2_score(Y_true, Y_predicted)
  meansquarederror = mean_squared_error(Y_true, Y_predicted)
  meanabsoluteerror = mean_absolute_error(Y_true, Y_predicted)
  maxerror = max_error(Y_true, Y_predicted)

  print('r2:',r2,'meansquarederror:',meansquarederror,'meanabsoluteerror:',meanabsoluteerror,'maxerror:',maxerror)
  return r2,meansquarederror,meanabsoluteerror,maxerror


# Random Forest Regressor

In [31]:
# database.iloc[[0,2,4,5,7,8]]

dataset_x,dataset_y = split_x_and_y(database.iloc[[0,1,2,3,4,5,6,7,8]],['A','E','N'],'Rt')

test_x,test_y = split_x_and_y(database.iloc[[9,10,11]],['A','E','N'],'Rt')

from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor(n_estimators=5000,random_state=100)
RFR.fit(dataset_x,dataset_y)
RFR.score(test_x,test_y)

0.3306427822593704

# SVM

In [9]:
dataset_x,dataset_y = split_x_and_y(database.iloc[[0,1,2,3,4,5,6,7,8]],['A','E','N'],'Rt')

test_x,test_y = split_x_and_y(database.iloc[[9,10,11]],['A','E','N'],'Rt')

from sklearn import svm

svr = svm.SVR(kernel='sigmoid',C=3,gamma='auto')

svr.fit(dataset_x,dataset_y)
svr.score(test_x,test_y)

0.4359749662202925

# MLP

In [29]:
dataset_x,dataset_y = split_x_and_y(database.iloc[[0,1,2,3,4,5,6,7,8]],['A','E','N'],'Rt')

test_x,test_y = split_x_and_y(database.iloc[[9,10,11]],['A','E','N'],'Rt')

from sklearn.neural_network import MLPRegressor
MLP = MLPRegressor(max_iter=1000,random_state=100)
MLP.fit(dataset_x,dataset_y)
MLP.score(test_x,test_y)

-0.05892459425244634

In [23]:
# import pickle
# pickle.dump(MLP, open('MLP_Rt.sav', 'wb'))

# Keras

In [32]:
import tensorflow as tf
from tensorflow import keras
# from tensorflow.python.keras.layers import Dense
import pandas as pd
from tensorflow.keras import layers

import keras
from keras.models import Sequential
from keras.layers import Dense

variables,results = split_x_and_y(database.iloc[[0,1,2,3,4,5,6,7,8]],['A','E','N'],'Rt')
test_x,test_y = split_x_and_y(database.iloc[[9,10,11]],['A','E','N'],'Rt')

model = keras.Sequential([
  layers.Dense(32, activation='sigmoid', input_shape=[len(variables.keys())]),
  layers.Dropout(0.2),
  layers.Dense(64, activation='relu'),
  # layers.Dense(64, activation='relu'),
  # layers.Dense(64, activation='relu'),
  layers.Dense(64, activation='relu'),
  # layers.Dropout(0.2),
  # layers.Dense(128, activation='relu'),
  # layers.Dense(64, activation='sigmoid'),
  # layers.Dense(8, activation='relu'),
  layers.Dense(1)
])

# optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

model.fit(variables.values,results.values,epochs=300)

Epoch 1/300
1/1 [==============================] - 0s 358ms/step - loss: 1.2673 - accuracy: 0.0000e+00
Epoch 2/300
1/1 [==============================] - 0s 3ms/step - loss: 1.2530 - accuracy: 0.0000e+00
Epoch 3/300
1/1 [==============================] - 0s 3ms/step - loss: 1.1787 - accuracy: 0.0000e+00
Epoch 4/300
1/1 [==============================] - 0s 3ms/step - loss: 1.1151 - accuracy: 0.0000e+00
Epoch 5/300
1/1 [==============================] - 0s 2ms/step - loss: 1.0840 - accuracy: 0.0000e+00
Epoch 6/300
1/1 [==============================] - 0s 3ms/step - loss: 1.0979 - accuracy: 0.0000e+00
Epoch 7/300
1/1 [==============================] - 0s 2ms/step - loss: 1.2371 - accuracy: 0.0000e+00
Epoch 8/300
1/1 [==============================] - 0s 3ms/step - loss: 1.1533 - accuracy: 0.0000e+00
Epoch 9/300
1/1 [==============================] - 0s 6ms/step - loss: 0.9981 - accuracy: 0.0000e+00
Epoch 10/300
1/1 [==============================] - 0s 5ms/step - loss: 1.1958 - accuracy

In [33]:
guess = model.predict(test_x.values)
desnormalizar = test_x.copy()
desnormalizar['Rt'] = guess
desnormalizado_teste = standardscaler.inverse_transform(desnormalizar)
desnormalizado_teste

1/1 [==============================] - 0s 46ms/step


array([[25.        ,  1.        ,  8.        ,  2.04648727],
       [25.        ,  2.        ,  5.        ,  2.40571648],
       [35.        ,  3.        , 11.        ,  2.1229644 ]])

In [34]:
desnormalizar = test_x.copy()
desnormalizar['Rt'] = test_y
desnormalizado_resultado = standardscaler.inverse_transform(desnormalizar)
desnormalizado_resultado

array([[25.   ,  1.   ,  8.   ,  1.946],
       [25.   ,  2.   ,  5.   ,  2.991],
       [35.   ,  3.   , 11.   ,  2.896]])

In [35]:
scores(desnormalizado_resultado[:,-1],desnormalizado_teste[:,-1])

r2: -0.42283228067086487 meansquarederror: 0.31674617954867906 meanabsoluteerror: 0.4862687981613711 maxerror: 0.7730356045118487


(-0.42283228067086487,
 0.31674617954867906,
 0.4862687981613711,
 0.7730356045118487)